In [33]:
import numpy as np
import plotly.graph_objs as go
import jsonlines
import plotly.express as px


In [3]:
datas = []
with jsonlines.open("rank_evaluate/promptway_own|targetlm_do_sample_False|append_label_length_-1.jsonl") as f:
    for line in f:
        datas.append(line)



In [4]:
ranks = []
for data in datas:
    ranks.append(data["rank"])

In [55]:
max_length = max(len(l) for l in ranks)
padded_lists_pad_zero = [np.pad(l, (0, max_length - len(l)), 'constant' , constant_values=0) for l in ranks]
padded_lists_pad_minus = [np.pad(l, (0, max_length - len(l)), 'constant', constant_values=-1) for l in ranks]
stacked_array_pad_zero = np.stack(padded_lists_pad_zero)
stacked_array_pad_minus = np.stack(padded_lists_pad_minus)

In [56]:
summed_array = np.mean(stacked_array_pad_zero, axis=0)
fig = go.Figure(data=go.Bar(x=np.arange(len(summed_array)), y=summed_array))
fig.update_layout(title='Sum Rank for each index', xaxis_title='Index', yaxis_title='Rank Average')
fig.show()


This figure tells that first position is the place where divergence happen seriously.

In [58]:
tmp = stacked_array_pad_zero
tmp[:,0] = 0
summed_array = np.mean(tmp, axis=0)
fig = go.Figure(data=go.Bar(x=np.arange(len(summed_array)), y=summed_array))
fig.update_layout(title='Sum Rank for each index', xaxis_title='Index', yaxis_title='Rank Average')
fig.show()

plots summation acorss all instances and plots starting on second position.
This also indicates that sampling woule help to jailbreak.

In [24]:
def find_first_valid_index(arr):
    for i, value in enumerate(arr):
        if value != 0 and value != -1:
            return i
    return np.inf  # 如果没有找到，则返回无穷大

In [25]:
first_valid_indices = np.array([find_first_valid_index(row) for row in stacked_array_pad_minus])

In [49]:
filtered_array = first_valid_indices[np.isfinite(first_valid_indices)].astype(np.int64)
# fig = px.histogram(filtered_array, nbins=30)
# fig.show()
filtered_array

array([0, 0, 0, ..., 0, 0, 0])

In [53]:
counts = np.bincount(filtered_array)
values = np.arange(len(counts))

# 使用Plotly绘制条形图
fig = go.Figure(data=go.Bar(x=values, y=counts))
fig.update_layout(title='Count of 0s, 1s, and 2s in Array (int64)', xaxis_title='Index where divergence firstly happen', yaxis_title='Count')
fig.show()

For those have divergence between target and generations, it plots the position where divergence happen firstly.
Different from previous plot as previous plot would plot the list like [1,1,1,1] so they have number at index = 1. In this plots, it only take the position
where divergence happen at the first time into account.


Need to compare with the token without adversarial prompts to see if the dangerous words lie in high rank.... so this could probably prove that safety is an deligent art and easily finetune would increase it's probability... Since they are in the high rank. 所以如果真的要safe，应该要让他们保持在相当低的位置。